In [1]:
GLOVE = '/content/gdrive/MyDrive/MC934/glove/glove.42B.300d.txt'

In [44]:
WORDS_PER_EXAMPLE = 210

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!git clone https://TOKEN@github.com/fake-news-deep-learning/fake-news-classifier.git

Cloning into 'fake-news-classifier'...
remote: Enumerating objects: 357, done.
remote: Counting objects: 100% (357/357), done.
remote: Compressing objects: 100% (209/209), done.
remote: Total 357 (delta 229), reused 268 (delta 144), pack-reused 0
Receiving objects: 100% (357/357), 37.53 MiB | 25.08 MiB/s, done.
Resolving deltas: 100% (229/229), done.


In [4]:
%cd /content/fake-news-classifier
!git checkout train_lstm
%cd src/

/content/fake-news-classifier
Branch 'train_lstm' set up to track remote branch 'train_lstm' from 'origin'.
Switched to a new branch 'train_lstm'
/content/fake-news-classifier/src


In [30]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/fake-news-deep-learning/fake-news-classifier
   ea4fe6b..9c99363  train_lstm -> origin/train_lstm
Updating ea4fe6b..9c99363
Fast-forward
 src/train.py | 5 +++--
 1 file changed, 3 insertions(+), 2 deletions(-)


In [5]:
import train

In [31]:
import importlib
importlib.reload(train)

<module 'train' from '/content/fake-news-classifier/src/train.py'>

In [15]:
tokenizer, word2veq = train.prepare_data(GLOVE)

100%|██████████| 699/699 [00:00<00:00, 321365.61it/s]

Fitting Tokenizer to dataset.


Corpus has 29903 words.
Removed rare words, which were 78.49% of corpus.
Succesfully removed 136 stopwords.
Mapping vocabulary with size 6296 to sequences
Parsing GLOVE


1917494it [02:18, 13803.97it/s]
100%|██████████| 6296/6296 [00:00<00:00, 374274.87it/s]

word2vec embeds have len 300
Translating vocabulary
Could not find 173 words in GLOVE.


In [45]:
model, history = train.lstm_driver(tokenizer, word2veq, epochs=40, length=WORDS_PER_EXAMPLE)

100%|██████████| 84/84 [00:00<00:00, 2026.58it/s]

Converting text from train set to sequences.
Converting text from valid set to sequences.


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_layer (Dropout)      (None, 210, 300)          0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 600)               1442400   
_________________________________________________________________
linear_layer (Dense)         (None, 1)                 601       
Total params: 1,443,001
Trainable params: 1,443,001
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/40
15/15 [==============================] - 5s 124ms/step - loss: 0.6299 - binary_accuracy: 0.6691 - true_positives: 119.6250 - true_negatives: 119.2500 - false_positives: 56.7500 - false_negatives: 58.3750 - val_loss: 0.6052 - val_binary_accuracy: 0.7024 - val_true_positives: 18.0000 - val_true_negatives: 41.0000 - val_false_positives: 0.0000e+00 - val_false

In [46]:
import json

In [47]:
history_dict = {}

for cat, items in history.history.items():
    items_ = [str(item) for item in items]
    history_dict[cat] = items_

In [48]:
with open('/content/baseline_history.json', 'w') as out_file:
  json.dump(history_dict, out_file, indent=4)

In [49]:
model.save('/content/')

INFO:tensorflow:Assets written to: /content/assets


INFO:tensorflow:Assets written to: /content/assets


# Avaliação do modelo no conjunto de teste

In [50]:
import numpy as np
from tqdm import tqdm

from word2vec import prepare_tokenizer, text_to_sequence
from utils import load_glove

In [51]:
def eval(model, tokenizer, word2seq):

    print(f'Converting text from test set to sequences.')
    with open(f'../data/processed/test.json') as input_file:
        dataset = json.load(input_file)

    test_x = []
    test_y = []

    for entry in tqdm(dataset.values()):

        sequence = text_to_sequence(
            entry['text'],
            tokenizer.word_index,
            word2seq,
            length=WORDS_PER_EXAMPLE,
            mode='lstm'
        )

        test_x.append(sequence)
        test_y.append(0 if entry['label'] == 'fake' else 1)

    test_y = np.asarray(test_y, dtype=np.float32)
    test_x = np.asarray(test_x, dtype=np.float32)

    results = model.evaluate(test_x, test_y, verbose=1, return_dict=True)

    return results

In [52]:
results = eval(model, tokenizer, word2veq)

100%|██████████| 140/140 [00:00<00:00, 1520.85it/s]

Converting text from test set to sequences.


5/5 [==============================] - 0s 31ms/step - loss: 0.5006 - binary_accuracy: 0.8643 - true_positives: 63.0000 - true_negatives: 58.0000 - false_positives: 7.0000 - false_negatives: 12.0000


In [53]:
results

{'binary_accuracy': 0.8642857074737549,
 'false_negatives': 12.0,
 'false_positives': 7.0,
 'loss': 0.5005921721458435,
 'true_negatives': 58.0,
 'true_positives': 63.0}

In [54]:
with open('/content/results.json', 'w') as results_file:
  json.dump(results, results_file, indent=4)